In [5]:
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
from __future__ import print_function
import numpy as np 
import os
import glob
import cv2
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras



In [0]:

class train:
  """ 
  :path_to_folder: string
  :name_image_train: string
  :name_mask_train: string
  :path_to_save_model: string
  :height: int
  :width: int
  :channel: int
  :feature_factor: int
  :number_of_layers: int
  :batch_size: int
  :epochs: int
  :learning_rate: float
  
  This class deals with training the model. It includes the two models: 6 layered and 10 layered. The number of features can be set manually by using the feature_factor 
  and also the learning rates can be changed. The constructor is explained below.
  """
  def __init__(self,path_to_folder,name_image_train,name_mask_train,path_to_save_model,\
               height=512,width=512,channel=1,feature_factor=64,number_of_layers=6,batch_size=2,epochs=5,learning_rate=5e-4):
    self.img_trn=name_image_train
    self.msk_trn=name_mask_train
    self.feature=feature_factor
    self.Image_height=height
    self.Image_width=width
    self.Image_channel=channel
    self.number_of_layer=number_of_layers
    self.batch=batch_size
    self.epoch=epochs
    self.path=path_to_folder
    self.save_model=path_to_save_model
    self.lr=learning_rate

    
  def load_data(self,path,name):
    return np.load(path+'/'+name)

  
  def ten_layer(self,feature_factor=64,pretrained_weights = None):
      '''
      :feature_factor: int
      :pretrained_weights: string(path to the weights/model)
      
      This generates the untrained 10 layered Unet model.
      '''
      inputs = Input((self.Image_height,self.Image_width,self.Image_channel))
      conv1 = Conv2D(feature_factor, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
      conv1 = Dropout(0.2)(conv1)
      conv1 = Conv2D(feature_factor, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
      conv1 = BatchNormalization()(conv1)
      pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
      conv2 = Conv2D(feature_factor*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)

      conv2 = Dropout(0.2)(conv2)
      conv2 = Conv2D(feature_factor*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
      conv2 = BatchNormalization()(conv2)
      pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
      conv3 = Conv2D(feature_factor*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
      conv3 = Dropout(0.2)(conv3)
      conv3 = Conv2D(feature_factor*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
      conv3 = BatchNormalization()(conv3)
      pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
      conv4 = Conv2D(feature_factor*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
      conv4 = Dropout(0.2)(conv4)
      conv4 = Conv2D(feature_factor*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
      conv4 = BatchNormalization()(conv4)
      drop4 = Dropout(0.5)(conv4)
      pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

      conv5 = Conv2D(feature_factor*16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
      conv5 = Conv2D(feature_factor*16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
      conv5 = BatchNormalization()(conv5)
      drop5 = Dropout(0.2)(conv5)

      up6 = Conv2D(feature_factor*8, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
      merge6 = concatenate([drop4,up6], axis = 3)
      conv6 = Conv2D(feature_factor*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
      conv6 = Dropout(0.2)(conv6)
      conv6 = Conv2D(feature_factor*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
      conv6 = BatchNormalization()(conv6)

      up7 = Conv2D(feature_factor*4, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
      merge7 = concatenate([conv3,up7], axis = 3)
      conv7 = Conv2D(feature_factor*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
      conv7 = Dropout(0.2)(conv7)
      conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
      conv7 = BatchNormalization()(conv7)

      up8 = Conv2D(feature_factor*2, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
      merge8 = concatenate([conv2,up8], axis = 3)
      conv8 = Conv2D(feature_factor*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
      conv8 = Dropout(0.2)(conv8)
      conv8 = Conv2D(feature_factor*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
      conv8 = BatchNormalization()(conv8)

      up9 = Conv2D(feature_factor, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
      merge9 = concatenate([conv1,up9], axis = 3)
      conv9 = Conv2D(feature_factor, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
      conv9 = Conv2D(feature_factor, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
      conv9 = BatchNormalization()(conv9)
      conv9 = Dropout(0.2)(conv9)
      conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
      conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

      model = Model(input = inputs, output = conv10)

      model.compile(optimizer = Adam(lr = 5e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

      model.summary()


      if(pretrained_weights):
        model.load_weights(pretrained_weights)

      return model

    
  def six_layer(self,feature_factor=64,pretrained_weights = None ): 
      '''
      :feature_factor: int
      :pretrained_weights: string (path to the weights/model)
      
      This generates the untrained 6 layered Unet model. 
      '''
      inputs = Input((self.Image_height,self.Image_width,self.Image_channel))
      conv1 = Conv2D(feature_factor, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
      conv1 = Dropout(0.2)(conv1)
      conv1 = Conv2D(feature_factor, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
      conv1 = BatchNormalization()(conv1)
      pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
      conv2 = Conv2D(feature_factor*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
      #conv2 = Dropout(0.2)(conv2)
      conv2 = Conv2D(feature_factor*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
      conv2 = BatchNormalization()(conv2)
      pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

      conv3 = Conv2D(feature_factor*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
      conv3 = Dropout(0.2)(conv3)
      conv3 = Conv2D(feature_factor*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
      conv3 = BatchNormalization()(conv3)


      up4 = Conv2D(feature_factor*2, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv3))
      merge4 = concatenate([conv2,up4],axis=3)
      conv4 = Conv2D(feature_factor*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge4)
      #conv4 = Dropout(0.2)(conv4)
      conv4 = Conv2D(feature_factor*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
      conv4 = BatchNormalization()(conv4)

      up5 = Conv2D(feature_factor, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv4))
      merge5 = concatenate([conv1,up5],axis=3)
      conv5 = Conv2D(feature_factor, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge5)
      conv5 = Conv2D(feature_factor, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
      conv5 = BatchNormalization()(conv5)
      conv5 = Dropout(0.2)(conv5)
      conv5 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)

      conv6 = Conv2D(1, 1, activation = 'sigmoid')(conv5)
      model = Model(input = inputs, output = conv6)

      model.compile(optimizer = Adam(lr = self.lr), loss = 'binary_crossentropy', metrics = ['accuracy'])

      model.summary()

      if(pretrained_weights):
        model.load_weights(pretrained_weights)

      return model

    
  def compile(self):
    '''
    :rtype: trained model
    
    This function calls the model and trains it with the given dataset. 
    We first load the data then initialise the model and then create a checkpoint to 
    save the model with the best results and then train the model with the checkpoint.
    '''
    img_trn=self.load_data(self.path,self.img_trn)
    msk_trn=self.load_data(self.path,self.msk_trn)
    if self.number_of_layer==6:
      model = self.six_layer(feature_factor=self.feature)
    elif self.number_of_layer==10:
      model=self.ten_layer(feature_factor=self.feature)
    model_checkpoint = ModelCheckpoint(self.save_model, monitor='loss',verbose=1, save_best_only=True)
    model.fit(img_trn,msk_trn,batch_size=self.batch,epochs=self.epoch,callbacks=[model_checkpoint])
    return model
      
  
  
class test:
    """
    :path_to_testdata: string
    :path_to_outputfolder: string
    :model: tensor object(trained model)
    :COLOR_DICT: dictionary
    :image_height: int
    :image_width: int
    :image_channel: int
    :batch_size: int
    
    This class includes all the functions needed for running the model on the test dataset and saving the results. The constructor is explained below.
    """

    def __init__(self,path_to_testdata, path_to_outputfolder, model,COLOR_DICT,image_height=512,image_width=512,image_channel=1,batch_size=15):
        self.COLOR_DICT=COLOR_DICT
        self.Image_height=image_height
        self.Image_width=image_width
        self.Image_channel=image_channel
        self.model=model
        self.fetch_path=path_to_testdata
        self.save_path=path_to_outputfolder
        
        self.batch=batch_size
    
    
    def load_data(self,path):
        return np.load(path)
    
    
    def labelVisualize(self,num_class,color_dict,img):
        img = img[:,:,0] if len(img.shape) == 3 else img
        img_out = np.zeros(img.shape + (3,))
        for i in range(num_class):
            img_out[img == i,:] = color_dict[i]
        return img_out / 255.
      
    
    def saveResult(self,save_path,npyfile,flag_multi_class = False,num_class = 2):
        for i,item in enumerate(npyfile):
            img = self.labelVisualize(num_class,self.COLOR_DICT,item) if flag_multi_class else item[:,:,0]
            cv2.imwrite(os.path.join(save_path+'/'+"unet_%d_predict.bmp"%i),img*255)

    
    def execute(self):
      '''
      This loads the test data, runs it through the trained model and saves the outputs.
      '''
      imgtst=self.load_data(self.fetch_path)
      results = self.model.predict(imgtst,batch_size=self.batch,verbose=1)
      self.saveResult(self.save_path,results)
      



In [8]:
'''
constructor: train parameters

path_to_folder='<path to the train folder which includes images and label>''
name_image_train='<name of the image file .npy>'
name_mask_train='<name of the mask file .npy>'
path_to_save_model='<path to save the model with the name of the name of the file.hdf5>'
height=512           #height of the image default set to 512 
width=512            #width of the image default set to 512
channel=1            #channels of the image default is set to 1
feature_factor=64    #deciding factors on number of features
number_of_layers=6   # choose between 10 and 6
batch_size=2         # batch size for training
epochs=5             #number of epochs for training
learning_rate=5e-4   #learning rate for learning. default 5e-4.




train('<path to the train folder which includes images and label>',"<name of the image file.npy>","<name of the maskfile.npy>",\
      "<the path where you want to save the model and name of the model>",image_height,image_width,image_channels,feature_factor,
       number of layers (6 or 10),batch_size,epochs, learningrate ) <.compile> to call the main function
'''



net=train(path_to_folder='drive/My Drive/GAN_seg/ADDN-master/data',name_image_train='new_img_trn.npy',\
          name_mask_train='new_msk_trn.npy',path_to_save_model='drive/My Drive/GAN_seg/unet.hdf5', batch_size=4,number_of_layers=10).compile()


W0808 21:56:07.204146 140242217576320 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0808 21:56:07.270922 140242217576320 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0808 21:56:07.294000 140242217576320 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0808 21:56:07.322692 140242217576320 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0808 21:56

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 512, 512, 64) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 512, 512, 64) 36928       dropout_1[0][0]                  
__________________________________________________________________________________________________
batch_norm

In [9]:
'''
constructor: test

path_to_testdata=             #whole path to the test image file not just folder
path_to_outputfolder=         #folder where you want to save data
model=                        #trained model
COLOR_DICT=                   #colour dictionary for different classes
image_height=512,             #height of the image. default set to 512
image_width=512,              #width of the image. default set to 512
image_channel=1               #number of channels of the image. default set to 1
batch_size=15                 #batch size for testing. default set to 15




test('<path to the test folder which includes images in the dataset>',"<the path where you want to save the prediction outputs of the model>",
       trained model,colour dictionary for the classes,image_height,image_width,image_channels,batch_size)
       <.execute> to call the main function
'''

Sky = [200,200,200]
Building = [0,0,0]
Pole = [192,192,128]
Road = [128,64,128]
Pavement = [60,40,222]
Tree = [128,128,0]
SignSymbol = [192,128,128]
Fence = [64,64,128]
Car = [64,0,128]
Pedestrian = [64,64,0]
Bicyclist = [0,128,192]
Unlabelled = [0,0,0]

COLOR_DICT = np.array([Sky, Building, Pole, Road, Pavement,
                          Tree, SignSymbol, Fence, Car, Pedestrian, Bicyclist, Unlabelled])




test(path_to_testdata='drive/My Drive/GAN_seg/ADDN-master/data/img_tst.npy',path_to_outputfolder="drive/My Drive/GAN_seg",\
     model=net,COLOR_DICT=COLOR_DICT,image_height=512,image_width=512,image_channel=1,batch_size=15).execute()

30/30 [==============================] - 19s 629ms/step
